In [87]:
from bs4 import BeautifulSoup

# Open the HTML file and read its content
with open('test.html', 'r', encoding='utf-8') as file:
    content = file.read()

# Parse the content with BeautifulSoup
soup = BeautifulSoup(content, 'lxml')

In [88]:
prices = []
bedrooms = []
bathrooms = []
addresses = []
date_listed = []
links = []

for item in soup.select('homes-property-tile'):
    # price, address, and date listed
    prices.append(item.select_one('h3.price').text.strip())
    addresses.append(item.select_one('.address').text.strip())
    date_listed.append(item.select_one('.dateArea').text.strip())
    
    # bedrooms
    bedroom_element = item.select_one('img[alt="Bedrooms"]')
    if bedroom_element:
        bedrooms.append(bedroom_element.find_next_sibling().text)
    else:
        bedrooms.append(None)
    
    # bathrooms
    bathroom_element = item.select_one('img[alt="Bathrooms"]')
    if bathroom_element:
        bathrooms.append(bathroom_element.find_next_sibling().text)
    else:
        bathrooms.append(None)

    # link
    link_element = item.select_one('a[_ngcontent-homes-app-c149]')
    if link_element:
        links.append(link_element['href'])
    else:
        links.append(None)


In [89]:
# prices
# bedrooms
# bathrooms
# addresses
# date_listed
# links

In [93]:
df = pd.DataFrame({
    'Address': addresses,
    'Price': prices,
    'Bedrooms': bedrooms,
    'Bathrooms': bathrooms,
    'Date Listed': date_listed,
    'link': links
})

df['Price'] = df['Price'].str.replace('$', '').str.replace(' per week', '').astype(float)

df['Bedrooms'] = pd.to_numeric(df['Bedrooms'], errors='coerce').astype('Int64')
df['Bathrooms'] = pd.to_numeric(df['Bathrooms'], errors='coerce').astype('Int64')

# if either or bedrooms is nan replace with 1(assume a studio or single room). if both are nan leave them as nan.
cond1 = df['Bedrooms'].isna() & df['Bathrooms'].notna()
df.loc[cond1, 'Bedrooms'] = 1

cond2 = df['Bathrooms'].isna() & df['Bedrooms'].notna()
df.loc[cond2, 'Bathrooms'] = 1

df.to_csv('Current_listings.csv', index=False)

C:\Users\sfra6\AppData\Local\Temp\ipykernel_8780\3226067118.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Price'] = df['Price'].str.replace('$', '').str.replace(' per week', '').astype(float)


In [91]:
df

,Address,Price,Bedrooms,Bathrooms,Date Listed,link
0,"196 Campbell Road, Greenlane",1300.0,5,3,Listed: Yesterday,/address/auckland/greenlane/196-campbell-road/...
1,"7C/1 Emily Place, City Centre",800.0,2,1,Listed: Yesterday,/address/auckland/auckland-central/7c-1-emily-...
2,"1105/19 Anzac Ave, City Centre",600.0,2,1,Listed: Yesterday,/address/auckland/auckland-central/1105-19-anz...
3,"28 Georgina Street, Freemans Bay",700.0,1,1,Listed: Yesterday,/address/auckland/freemans-bay/28-georgina-str...
4,"5A Matai Road, Greenlane",660.0,2,1,Listed: Yesterday,/address/auckland/greenlane/5a-matai-road/aaOq5
...,...,...,...,...,...,...
926,"Furneaux Way, Remuera",1000.0,2,1,Listed: 03 Oct 2022,/address/auckland-city/remuera/furneaux-way/XK...
927,"192B Manukau Road, Epsom",370.0,1,1,Listed: 11 Sep 2022,/address/auckland/epsom/196-manukau-road/O0N0w
928,"Epsom, Auckland City",1400.0,4,4,Listed: 08 Aug 2022,/address/auckland-city/epsom/0/9lELQQ
929,"Mount Eden, Auckland City",500.0,2,1,Listed: 04 Aug 2022,/address/auckland/mount-eden/12-62-grange-road...
